In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import scipy.stats as ss # For running statistical functions 
import sklearn.neighbors as sn
import random 
import sklearn.metrics as sm

## STEP 1: Reading in the data set

In [2]:
netflix = pd.read_csv("C:/Users/micha/Documents/MyDatasets/netflix_title.csv")
netflix

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,14-Aug-20,2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,23-Dec-16,2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,20-Dec-18,2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,16-Nov-17,2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,01-Jan-20,2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...",19-Oct-20,2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,02-Mar-19,2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,25-Sep-20,2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,31-Oct-20,2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...


# STEP 2: Getting The Lead Directors + Headlining Cast Member

In [3]:
netflix['lead_director']= netflix['director'].str.split(",", expand = True).rename(columns = {0: "lead_director"}).iloc[:,0]
netflix.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,lead_director
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,14-Aug-20,2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,NaN
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,23-Dec-16,2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,Jorge Michel Grau


In [4]:
netflix['cast_lead']= netflix['cast'].str.split(",", expand = True).iloc[:,0]
netflix.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,lead_director,cast_lead
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,14-Aug-20,2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,NaN,João Miguel
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,23-Dec-16,2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,Jorge Michel Grau,Demián Bichir


In [5]:
netflix = netflix[['show_id','type','title','lead_director','cast_lead','country','release_year','rating','listed_in','description']]
netflix.head(2)

,show_id,type,title,lead_director,cast_lead,country,release_year,rating,listed_in,description
0,s1,TV Show,3%,NaN,João Miguel,Brazil,2020,TV-MA,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,Demián Bichir,Mexico,2016,TV-MA,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...


# STEP 3: Creating a variable to differentiate modern movies based on release year

In [6]:
netflix['is_modern'] = netflix['release_year'].apply(lambda x: 1 if x > 2000 else 0)
netflix = netflix[['show_id', 'type', "title", "lead_director", 'cast_lead', 'country', "is_modern", "rating", 'listed_in', 'description']]

C:\Users\micha\anaconda3.0\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,show_id,type,title,lead_director,cast_lead,country,is_modern,rating,listed_in,description
0,s1,TV Show,3%,NaN,João Miguel,Brazil,1,TV-MA,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,Demián Bichir,Mexico,1,TV-MA,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,Tedd Chan,Singapore,1,R,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,Elijah Wood,United States,1,PG-13,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."


# STEP 4: Differentiate b/t Movie and TV Show in a binary 

In [7]:
netflix['is_movie'] = netflix['type'].apply(lambda x: 1 if x == "Movie" else 0)
netflix = netflix[["show_id", "is_movie", "title", 'lead_director', 'cast_lead', "country", 'is_modern', "rating", 'listed_in', "description"]]

,show_id,is_movie,title,lead_director,cast_lead,country,is_modern,rating,listed_in,description
0,s1,0,3%,NaN,João Miguel,Brazil,1,TV-MA,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,1,7:19,Jorge Michel Grau,Demián Bichir,Mexico,1,TV-MA,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,1,23:59,Gilbert Chan,Tedd Chan,Singapore,1,R,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,1,9,Shane Acker,Elijah Wood,United States,1,PG-13,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,1,21,Robert Luketic,Jim Sturgess,United States,1,PG-13,Dramas,A brilliant group of students become card-coun...
...,...,...,...,...,...,...,...,...,...,...
7782,s7783,1,Zozo,Josef Fares,Imad Creidi,"Sweden, Czech Republic, United Kingdom, Denmar...",1,TV-MA,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,1,Zubaan,Mozez Singh,Vicky Kaushal,India,1,TV-14,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,1,Zulu Man in Japan,NaN,Nasty C,NaN,1,TV-MA,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,0,Zumbo's Just Desserts,NaN,Adriano Zumbo,Australia,1,TV-PG,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...


# STEP 5: Figuring out if a content is English or not

The approach here will be based on three columns: listed_in, title and country. 

Priority will be given to listed_in since it has a distinct differentiation that anything listing anime, korean, or spanish = non-English.  Every other genre is pretty much up in the air that needs to be further flushed out by the title for presence of non-ASCII characters + country where anything not listing English speaking = Foreign.

In [8]:
netflix['country'] = netflix['country'].replace(np.nan, "Unknown")

In [9]:
list_of_countries = pd.DataFrame(netflix['country'].str.split(",", expand = True)).rename(columns = {0:"country_1", 1:"country_2", 2:"country_3", 3:"country_4", 4:"country_5", 5:"country_6", 6:"country_7", 7:"country_8", 8:"country_9", 9:"country_10", 10:"country_11", 11:"country_12"})

In [10]:
list_of_countries = list_of_countries.select_dtypes(['object']).apply(lambda x: x.str.strip())
list_of_countries

,country_1,country_2,country_3,country_4,country_5,country_6,country_7,country_8,country_9,country_10,country_11,country_12
0,Brazil,None,None,None,None,None,None,None,None,None,None,None
1,Mexico,None,None,None,None,None,None,None,None,None,None,None
2,Singapore,None,None,None,None,None,None,None,None,None,None,None
3,United States,None,None,None,None,None,None,None,None,None,None,None
4,United States,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
7782,Sweden,Czech Republic,United Kingdom,Denmark,Netherlands,None,None,None,None,None,None,None
7783,India,None,None,None,None,None,None,None,None,None,None,None
7784,Unknown,None,None,None,None,None,None,None,None,None,None,None
7785,Australia,None,None,None,None,None,None,None,None,None,None,None


In [11]:
english_countries = "United States", "United Kingdom", "Ireland", "Canada", "Australia", "New Zealand", "Jamaica"

list_of_countries['english_check_1'] = list_of_countries['country_1'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_2'] = list_of_countries['country_2'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_3'] = list_of_countries['country_3'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_4'] = list_of_countries['country_4'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_5'] = list_of_countries['country_5'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_6'] = list_of_countries['country_6'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_7'] = list_of_countries['country_7'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_8'] = list_of_countries['country_8'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_9'] = list_of_countries['country_9'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_10'] = list_of_countries['country_10'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_11'] = list_of_countries['country_11'].apply(lambda x: "Yes" if x in english_countries else "no")
list_of_countries['english_check_12'] = list_of_countries['country_12'].apply(lambda x: "Yes" if x in english_countries else "no")

list_of_countries

,country_1,country_2,country_3,country_4,country_5,country_6,country_7,country_8,country_9,country_10,...,english_check_3,english_check_4,english_check_5,english_check_6,english_check_7,english_check_8,english_check_9,english_check_10,english_check_11,english_check_12
0,Brazil,None,None,None,None,None,None,None,None,None,...,no,no,no,no,no,no,no,no,no,no
1,Mexico,None,None,None,None,None,None,None,None,None,...,no,no,no,no,no,no,no,no,no,no
2,Singapore,None,None,None,None,None,None,None,None,None,...,no,no,no,no,no,no,no,no,no,no
3,United States,None,None,None,None,None,None,None,None,None,...,no,no,no,no,no,no,no,no,no,no
4,United States,None,None,None,None,None,None,None,None,None,...,no,no,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7782,Sweden,Czech Republic,United Kingdom,Denmark,Netherlands,None,None,None,None,None,...,Yes,no,no,no,no,no,no,no,no,no
7783,India,None,None,None,None,None,None,None,None,None,...,no,no,no,no,no,no,no,no,no,no
7784,Unknown,None,None,None,None,None,None,None,None,None,...,no,no,no,no,no,no,no,no,no,no
7785,Australia,None,None,None,None,None,None,None,None,None,...,no,no,no,no,no,no,no,no,no,no


In [12]:
netflix = pd.concat([netflix,pd.DataFrame(list_of_countries.iloc[:,12:].select_dtypes("object").apply(lambda row: row.str.contains("Yes"), axis = 1).any(axis = 1))], axis = 1)
netflix = netflix.rename(columns = {0:"is_english"})

In [13]:
new_netflix = netflix[['show_id', 'title', 'is_movie', 'lead_director', 'cast_lead', 'is_english', 'is_modern', 'rating', 'listed_in', 'description']]
new_netflix.head(10)

,show_id,title,is_movie,lead_director,cast_lead,is_english,is_modern,rating,listed_in,description
0,s1,3%,0,NaN,João Miguel,False,1,TV-MA,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,7:19,1,Jorge Michel Grau,Demián Bichir,False,1,TV-MA,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,23:59,1,Gilbert Chan,Tedd Chan,False,1,R,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,9,1,Shane Acker,Elijah Wood,True,1,PG-13,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,21,1,Robert Luketic,Jim Sturgess,True,1,PG-13,Dramas,A brilliant group of students become card-coun...
5,s6,46,0,Serdar Akar,Erdal Beşikçioğlu,False,1,TV-MA,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...
6,s7,122,1,Yasir Al Yasiri,Amina Khalil,False,1,TV-MA,"Horror Movies, International Movies","After an awful accident, a couple admitted to ..."
7,s8,187,1,Kevin Reynolds,Samuel L. Jackson,True,0,R,Dramas,After one of his high school students attacks ...
8,s9,706,1,Shravan Kumar,Divya Dutta,False,1,TV-14,"Horror Movies, International Movies","When a doctor goes missing, his psychiatrist w..."
9,s10,1920,1,Vikram Bhatt,Rajneesh Duggal,False,1,TV-MA,"Horror Movies, International Movies, Thrillers",An architect and his wife move into a castle t...


# STEP 6: Blow-up the genre + rating

In [14]:
list_of_genres = pd.DataFrame(netflix['listed_in'].str.split(",", expand = True)).rename(columns = {0:"genre_1", 1:"genre_2", 2:"genre_3"})
list_of_genres = list_of_genres.select_dtypes(['object']).apply(lambda x: x.str.strip())
list_of_genres.head(800)

,genre_1,genre_2,genre_3
0,International TV Shows,TV Dramas,TV Sci-Fi & Fantasy
1,Dramas,International Movies,None
2,Horror Movies,International Movies,None
3,Action & Adventure,Independent Movies,Sci-Fi & Fantasy
4,Dramas,None,None
...,...,...,...
795,Comedies,None,None
796,Comedies,Dramas,International Movies
797,International TV Shows,Korean TV Shows,Romantic TV Shows
798,Comedies,International Movies,None


In [15]:
list_of_genres['genre_1'].unique()

array(['International TV Shows', 'Dramas', 'Horror Movies',
       'Action & Adventure', 'Crime TV Shows', 'Documentaries',
       'Independent Movies', 'Comedies', 'Sports Movies', 'Anime Series',
       'Reality TV', 'TV Comedies', 'Docuseries', 'Movies',
       'British TV Shows', 'International Movies', 'Sci-Fi & Fantasy',
       "Kids' TV", 'Children & Family Movies', 'TV Shows',
       'Classic Movies', 'Thrillers', 'Stand-Up Comedy & Talk Shows',
       'Stand-Up Comedy', 'Anime Features', 'Music & Musicals',
       'TV Dramas', 'TV Horror', 'TV Action & Adventure',
       'Classic & Cult TV', 'Romantic TV Shows', 'Cult Movies',
       'TV Sci-Fi & Fantasy', 'Romantic Movies',
       'Spanish-Language TV Shows', 'LGBTQ Movies'], dtype=object)

In [16]:
international = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("International Movies|International TV Shows|British TV Shows|Korean TV Shows|Spanish-Language TV Shows"), axis = 1).any(axis = 1)).rename(columns = {0:"is_international"})
drama = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Dramas|TV Dramas"), axis = 1).any(axis = 1)).rename(columns = {0:"is_drama"})
horror = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Horror Movies|TV Horror"), axis = 1).any(axis = 1)).rename(columns = {0:"is_horror"})
action_adventure = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Action & Adventure|TV Action & Adventure"), axis = 1).any(axis = 1)).rename(columns = {0:"is_action_adventure"})
crime_mystery_thrillers = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Crime TV Shows|Thrillers|TV Thrillers|TV Mysteries"), axis = 1).any(axis = 1)).rename(columns = {0:"is_crime_mystery_thrillers"})
docu = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Documentaries|Docuseries"), axis = 1).any(axis = 1)).rename(columns = {0:"is_docu"})
indie = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Independent Movies"), axis = 1).any(axis = 1)).rename(columns = {0:"is_indie"})
reality = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Reality TV"), axis = 1).any(axis = 1)).rename(columns = {0:"is_reality"})
comedy = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("TV Comedies|Comedies|Stand-Up Comedy|Stand-Up Comedy & Talk Shows"), axis = 1).any(axis = 1)).rename(columns = {0:"is_comedy"})
anime = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Anime Series|Anime Features"), axis = 1).any(axis = 1)).rename(columns = {0:"is_anime"})
sports = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Sports Movies"), axis = 1).any(axis = 1)).rename(columns = {0:"is_sports"})
sci_fi = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("TV Sci-Fi & Fantasy|Sci-Fi & Fantasy"), axis = 1).any(axis = 1)).rename(columns = {0:"is_sci_fi"})
musicals = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Music & Musicals"), axis = 1).any(axis = 1)).rename(columns = {0:"is_musical"})
family = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Kids' TV|Children & Family Movies|Teen TV Shows|Faith & Spirituality"), axis = 1).any(axis = 1)).rename(columns = {0:"is_family"})
classic_cult = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Classic Movies|Cult Movies|Classic & Cult TV"), axis = 1).any(axis = 1)).rename(columns = {0:"is_classic"})
romantic = pd.DataFrame(list_of_genres.select_dtypes("object").apply(lambda row: row.str.contains("Romantic TV Shows|Romantic Movies|LGBTQ Movies"), axis = 1).any(axis = 1)).rename(columns = {0:"is_romantic"})

In [17]:
new_netflix = pd.concat([new_netflix, international, drama, horror, action_adventure, crime_mystery_thrillers, docu, indie, reality, comedy, anime, sports, sci_fi, musicals, family, classic_cult, romantic], axis = 1)
new_netflix = new_netflix[['show_id', 'title', 'is_movie', 'lead_director', 'cast_lead','is_english', 'is_modern', 'rating', 'description', 'is_international', 'is_drama', 'is_horror', 'is_action_adventure', 'is_crime_mystery_thrillers', 'is_docu', 'is_indie', 'is_reality','is_comedy', 'is_anime', 'is_sports', 'is_sci_fi', 'is_musical', 'is_family', 'is_classic', 'is_romantic']]
new_netflix.head(3)

,show_id,title,is_movie,lead_director,cast_lead,is_english,is_modern,rating,description,is_international,...,is_indie,is_reality,is_comedy,is_anime,is_sports,is_sci_fi,is_musical,is_family,is_classic,is_romantic
0,s1,3%,0,NaN,João Miguel,False,1,TV-MA,In a future where the elite inhabit an island ...,True,...,False,False,False,False,False,True,False,False,False,False
1,s2,7:19,1,Jorge Michel Grau,Demián Bichir,False,1,TV-MA,After a devastating earthquake hits Mexico Cit...,True,...,False,False,False,False,False,False,False,False,False,False
2,s3,23:59,1,Gilbert Chan,Tedd Chan,False,1,R,"When an army recruit is found dead, his fellow...",True,...,False,False,False,False,False,False,False,False,False,False


In [18]:
netflix['rating'].unique()

array(['TV-MA', 'R', 'PG-13', 'TV-14', 'TV-PG', 'NR', 'TV-G', 'TV-Y', nan,
       'TV-Y7', 'PG', 'G', 'NC-17', 'TV-Y7-FV', 'UR'], dtype=object)

In [19]:
new_netflix["is_TV_MA"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "TV-MA" else 0))
new_netflix["is_R"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "R" else 0))
new_netflix["is_PG_13"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "PG-13" else 0))
new_netflix["is_TV_14"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "TV-14" else 0))
new_netflix["is_TV_PG"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "TV-PG" else 0))
new_netflix["is_PG"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "PG" else 0))
new_netflix["is_TV_G"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "TV-G" else 0))
new_netflix["is_TV_Y"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "TV-Y" else 0))
new_netflix["is_TV_Y7"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "TV-Y7" else (1 if x == "TV-Y7-FV" else 0)))
new_netflix["is_G"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "G" else 0))
new_netflix["is_NC_17"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "NC-17" else 0))
new_netflix["is_unrated"] = pd.DataFrame(netflix['rating'].apply(lambda x: 1 if x == "NR" else (1 if x == "UR" else 0)))
new_netflix.head(4)

,show_id,title,is_movie,lead_director,cast_lead,is_english,is_modern,rating,description,is_international,...,is_PG_13,is_TV_14,is_TV_PG,is_PG,is_TV_G,is_TV_Y,is_TV_Y7,is_G,is_NC_17,is_unrated
0,s1,3%,0,NaN,João Miguel,False,1,TV-MA,In a future where the elite inhabit an island ...,True,...,0,0,0,0,0,0,0,0,0,0
1,s2,7:19,1,Jorge Michel Grau,Demián Bichir,False,1,TV-MA,After a devastating earthquake hits Mexico Cit...,True,...,0,0,0,0,0,0,0,0,0,0
2,s3,23:59,1,Gilbert Chan,Tedd Chan,False,1,R,"When an army recruit is found dead, his fellow...",True,...,0,0,0,0,0,0,0,0,0,0
3,s4,9,1,Shane Acker,Elijah Wood,True,1,PG-13,"In a postapocalyptic world, rag-doll robots hi...",False,...,1,0,0,0,0,0,0,0,0,0


In [20]:
new_netflix.columns

Index(['show_id', 'title', 'is_movie', 'lead_director', 'cast_lead',
       'is_english', 'is_modern', 'rating', 'description', 'is_international',
       'is_drama', 'is_horror', 'is_action_adventure',
       'is_crime_mystery_thrillers', 'is_docu', 'is_indie', 'is_reality',
       'is_comedy', 'is_anime', 'is_sports', 'is_sci_fi', 'is_musical',
       'is_family', 'is_classic', 'is_romantic', 'is_TV_MA', 'is_R',
       'is_PG_13', 'is_TV_14', 'is_TV_PG', 'is_PG', 'is_TV_G', 'is_TV_Y',
       'is_TV_Y7', 'is_G', 'is_NC_17', 'is_unrated'],
      dtype='object')

In [21]:
new_netflix = new_netflix[['show_id', 'title', 'is_movie', 'lead_director', 'cast_lead','is_english', 'is_modern', 'description', 'is_international', 'is_drama', 'is_horror','is_action_adventure', 'is_crime_mystery_thrillers', 'is_docu', 'is_indie', 'is_reality', 'is_comedy', 'is_anime', 'is_sports', 'is_sci_fi', 'is_musical', 'is_family', 'is_classic', 'is_romantic', 'is_TV_MA', 'is_R', 'is_PG_13', 'is_TV_14', 'is_TV_PG', 'is_PG', 'is_TV_G', 'is_TV_Y', 'is_TV_Y7', 'is_G', 'is_NC_17','is_unrated']]
new_netflix

,show_id,title,is_movie,lead_director,cast_lead,is_english,is_modern,description,is_international,is_drama,...,is_PG_13,is_TV_14,is_TV_PG,is_PG,is_TV_G,is_TV_Y,is_TV_Y7,is_G,is_NC_17,is_unrated
0,s1,3%,0,NaN,João Miguel,False,1,In a future where the elite inhabit an island ...,True,True,...,0,0,0,0,0,0,0,0,0,0
1,s2,7:19,1,Jorge Michel Grau,Demián Bichir,False,1,After a devastating earthquake hits Mexico Cit...,True,True,...,0,0,0,0,0,0,0,0,0,0
2,s3,23:59,1,Gilbert Chan,Tedd Chan,False,1,"When an army recruit is found dead, his fellow...",True,False,...,0,0,0,0,0,0,0,0,0,0
3,s4,9,1,Shane Acker,Elijah Wood,True,1,"In a postapocalyptic world, rag-doll robots hi...",False,False,...,1,0,0,0,0,0,0,0,0,0
4,s5,21,1,Robert Luketic,Jim Sturgess,True,1,A brilliant group of students become card-coun...,False,True,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Zozo,1,Josef Fares,Imad Creidi,True,1,When Lebanon's Civil War deprives Zozo of his ...,True,True,...,0,0,0,0,0,0,0,0,0,0
7783,s7784,Zubaan,1,Mozez Singh,Vicky Kaushal,False,1,A scrappy but poor boy worms his way into a ty...,True,True,...,0,1,0,0,0,0,0,0,0,0
7784,s7785,Zulu Man in Japan,1,NaN,Nasty C,False,1,"In this documentary, South African rapper Nast...",True,False,...,0,0,0,0,0,0,0,0,0,0
7785,s7786,Zumbo's Just Desserts,0,NaN,Adriano Zumbo,True,1,Dessert wizard Adriano Zumbo looks for the nex...,True,False,...,0,0,1,0,0,0,0,0,0,0


In [22]:
new_netflix_update = new_netflix.copy()
new_netflix_update.index = new_netflix['title']

In [23]:
new_netflix_update = new_netflix_update[['show_id', 'is_movie', 'lead_director', 'cast_lead', 'is_english', 'is_modern', 'description', 'is_international', 'is_drama', 'is_horror', 'is_action_adventure', 'is_crime_mystery_thrillers', 'is_docu', 'is_indie', 'is_reality', 'is_comedy', 'is_anime', 'is_sports', 'is_sci_fi', 'is_musical', 'is_family', 'is_classic', 'is_romantic', 'is_TV_MA', 'is_R', 'is_PG_13', 'is_TV_14', 'is_TV_PG', 'is_PG', 'is_TV_G', 'is_TV_Y', 'is_TV_Y7', 'is_G', 'is_NC_17', 'is_unrated']]
new_netflix_update.head(10)

,show_id,is_movie,lead_director,cast_lead,is_english,is_modern,description,is_international,is_drama,is_horror,...,is_PG_13,is_TV_14,is_TV_PG,is_PG,is_TV_G,is_TV_Y,is_TV_Y7,is_G,is_NC_17,is_unrated
title,,,,,,,,,,,,,,,,,,,,,
3%,s1,0,NaN,João Miguel,False,1,In a future where the elite inhabit an island ...,True,True,False,...,0,0,0,0,0,0,0,0,0,0
7:19,s2,1,Jorge Michel Grau,Demián Bichir,False,1,After a devastating earthquake hits Mexico Cit...,True,True,False,...,0,0,0,0,0,0,0,0,0,0
23:59,s3,1,Gilbert Chan,Tedd Chan,False,1,"When an army recruit is found dead, his fellow...",True,False,True,...,0,0,0,0,0,0,0,0,0,0
9,s4,1,Shane Acker,Elijah Wood,True,1,"In a postapocalyptic world, rag-doll robots hi...",False,False,False,...,1,0,0,0,0,0,0,0,0,0
21,s5,1,Robert Luketic,Jim Sturgess,True,1,A brilliant group of students become card-coun...,False,True,False,...,1,0,0,0,0,0,0,0,0,0
46,s6,0,Serdar Akar,Erdal Beşikçioğlu,False,1,A genetics professor experiments with a treatm...,True,True,False,...,0,0,0,0,0,0,0,0,0,0
122,s7,1,Yasir Al Yasiri,Amina Khalil,False,1,"After an awful accident, a couple admitted to ...",True,False,True,...,0,0,0,0,0,0,0,0,0,0
187,s8,1,Kevin Reynolds,Samuel L. Jackson,True,0,After one of his high school students attacks ...,False,True,False,...,0,0,0,0,0,0,0,0,0,0
706,s9,1,Shravan Kumar,Divya Dutta,False,1,"When a doctor goes missing, his psychiatrist w...",True,False,True,...,0,1,0,0,0,0,0,0,0,0


In [24]:
new_netflix_update['is_english'] = new_netflix_update['is_english'].apply(lambda x: 0 if x == False else 1)
new_netflix_update.head(3)

,show_id,is_movie,lead_director,cast_lead,is_english,is_modern,description,is_international,is_drama,is_horror,...,is_PG_13,is_TV_14,is_TV_PG,is_PG,is_TV_G,is_TV_Y,is_TV_Y7,is_G,is_NC_17,is_unrated
title,,,,,,,,,,,,,,,,,,,,,
3%,s1,0,NaN,João Miguel,0,1,In a future where the elite inhabit an island ...,True,True,False,...,0,0,0,0,0,0,0,0,0,0
7:19,s2,1,Jorge Michel Grau,Demián Bichir,0,1,After a devastating earthquake hits Mexico Cit...,True,True,False,...,0,0,0,0,0,0,0,0,0,0
23:59,s3,1,Gilbert Chan,Tedd Chan,0,1,"When an army recruit is found dead, his fellow...",True,False,True,...,0,0,0,0,0,0,0,0,0,0


In [25]:
new_netflix_update['is_international'] = new_netflix_update['is_international'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_drama'] = new_netflix_update['is_drama'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_horror'] = new_netflix_update['is_horror'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_action_adventure'] = new_netflix_update['is_action_adventure'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_crime_mystery_thrillers'] = new_netflix_update['is_crime_mystery_thrillers'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_docu'] = new_netflix_update['is_docu'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_indie'] = new_netflix_update['is_indie'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_reality'] = new_netflix_update['is_reality'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_comedy'] = new_netflix_update['is_comedy'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_anime'] = new_netflix_update['is_anime'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_sports'] = new_netflix_update['is_sports'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_sci_fi'] = new_netflix_update['is_sci_fi'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_musical'] = new_netflix_update['is_musical'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_family'] = new_netflix_update['is_family'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_classic'] = new_netflix_update['is_classic'].apply(lambda x: 1 if x == True else 0)
new_netflix_update['is_romantic'] = new_netflix_update['is_romantic'].apply(lambda x: 1 if x == True else 0)
new_netflix_update

,show_id,is_movie,lead_director,cast_lead,is_english,is_modern,description,is_international,is_drama,is_horror,...,is_PG_13,is_TV_14,is_TV_PG,is_PG,is_TV_G,is_TV_Y,is_TV_Y7,is_G,is_NC_17,is_unrated
title,,,,,,,,,,,,,,,,,,,,,
3%,s1,0,NaN,João Miguel,0,1,In a future where the elite inhabit an island ...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
7:19,s2,1,Jorge Michel Grau,Demián Bichir,0,1,After a devastating earthquake hits Mexico Cit...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
23:59,s3,1,Gilbert Chan,Tedd Chan,0,1,"When an army recruit is found dead, his fellow...",1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,s4,1,Shane Acker,Elijah Wood,1,1,"In a postapocalyptic world, rag-doll robots hi...",0,0,0,...,1,0,0,0,0,0,0,0,0,0
21,s5,1,Robert Luketic,Jim Sturgess,1,1,A brilliant group of students become card-coun...,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zozo,s7783,1,Josef Fares,Imad Creidi,1,1,When Lebanon's Civil War deprives Zozo of his ...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Zubaan,s7784,1,Mozez Singh,Vicky Kaushal,0,1,A scrappy but poor boy worms his way into a ty...,1,1,0,...,0,1,0,0,0,0,0,0,0,0
Zulu Man in Japan,s7785,1,NaN,Nasty C,0,1,"In this documentary, South African rapper Nast...",1,0,0,...,0,0,0,0,0,0,0,0,0,0


# STEP 7: Last final fixes with missing entries for director + lead cast

In [26]:
new_netflix_update['lead_director'] = new_netflix_update['lead_director'].fillna("Unknown Director")
new_netflix_update['cast_lead'] = new_netflix_update['cast_lead'].fillna("Unknown Cast")
new_netflix_update.head(20)

,show_id,is_movie,lead_director,cast_lead,is_english,is_modern,description,is_international,is_drama,is_horror,...,is_PG_13,is_TV_14,is_TV_PG,is_PG,is_TV_G,is_TV_Y,is_TV_Y7,is_G,is_NC_17,is_unrated
title,,,,,,,,,,,,,,,,,,,,,
3%,s1,0,Unknown Director,João Miguel,0,1,In a future where the elite inhabit an island ...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
7:19,s2,1,Jorge Michel Grau,Demián Bichir,0,1,After a devastating earthquake hits Mexico Cit...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
23:59,s3,1,Gilbert Chan,Tedd Chan,0,1,"When an army recruit is found dead, his fellow...",1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,s4,1,Shane Acker,Elijah Wood,1,1,"In a postapocalyptic world, rag-doll robots hi...",0,0,0,...,1,0,0,0,0,0,0,0,0,0
21,s5,1,Robert Luketic,Jim Sturgess,1,1,A brilliant group of students become card-coun...,0,1,0,...,1,0,0,0,0,0,0,0,0,0
46,s6,0,Serdar Akar,Erdal Beşikçioğlu,0,1,A genetics professor experiments with a treatm...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
122,s7,1,Yasir Al Yasiri,Amina Khalil,0,1,"After an awful accident, a couple admitted to ...",1,0,1,...,0,0,0,0,0,0,0,0,0,0
187,s8,1,Kevin Reynolds,Samuel L. Jackson,1,0,After one of his high school students attacks ...,0,1,0,...,0,0,0,0,0,0,0,0,0,0
706,s9,1,Shravan Kumar,Divya Dutta,0,1,"When a doctor goes missing, his psychiatrist w...",1,0,1,...,0,1,0,0,0,0,0,0,0,0


In [27]:
new_netflix_update_knn = new_netflix_update[['is_movie', 'lead_director', 'cast_lead', 'is_english',
       'is_modern', 'is_international', 'is_drama', 'is_horror',
       'is_action_adventure', 'is_crime_mystery_thrillers', 'is_docu',
       'is_indie', 'is_reality', 'is_comedy', 'is_anime', 'is_sports',
       'is_sci_fi', 'is_musical', 'is_family', 'is_classic', 'is_romantic',
       'is_TV_MA', 'is_R', 'is_PG_13', 'is_TV_14', 'is_TV_PG', 'is_PG',
       'is_TV_G', 'is_TV_Y', 'is_TV_Y7', 'is_G', 'is_NC_17', 'is_unrated']]

In [28]:
new_netflix_update_knn

,is_movie,lead_director,cast_lead,is_english,is_modern,is_international,is_drama,is_horror,is_action_adventure,is_crime_mystery_thrillers,...,is_PG_13,is_TV_14,is_TV_PG,is_PG,is_TV_G,is_TV_Y,is_TV_Y7,is_G,is_NC_17,is_unrated
title,,,,,,,,,,,,,,,,,,,,,
3%,0,Unknown Director,João Miguel,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7:19,1,Jorge Michel Grau,Demián Bichir,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23:59,1,Gilbert Chan,Tedd Chan,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,Shane Acker,Elijah Wood,1,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
21,1,Robert Luketic,Jim Sturgess,1,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zozo,1,Josef Fares,Imad Creidi,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zubaan,1,Mozez Singh,Vicky Kaushal,0,1,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
Zulu Man in Japan,1,Unknown Director,Nasty C,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
dummified_cast = pd.get_dummies(new_netflix_update_knn['cast_lead'])
dummified_director = pd.get_dummies(new_netflix_update_knn['lead_director'])

In [30]:
dummified_director

,A. L. Vijay,A. Raajdheep,A. Salaam,A.R. Murugadoss,Aadish Keluskar,Aamir Bashir,Aamir Khan,Aanand Rai,Aaron Burns,Aaron Hancox,...,Álex de la Iglesia,Álvaro Brechner,Álvaro Delgado-Aparicio L.,Álvaro Longoria,Ángel Gómez Hernández,Çagan Irmak,Ísold Uggadóttir,Óskar Thór Axelsson,Ömer Faruk Sorak,Şenol Sönmez
title,,,,,,,,,,,,,,,,,,,,,
3%,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7:19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23:59,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zozo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zubaan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zulu Man in Japan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
newer_netflix_update_knn = new_netflix_update_knn[['is_movie','is_english', 'is_modern',
       'is_international', 'is_drama', 'is_horror', 'is_action_adventure',
       'is_crime_mystery_thrillers', 'is_docu', 'is_indie', 'is_reality',
       'is_comedy', 'is_anime', 'is_sports', 'is_sci_fi', 'is_musical',
       'is_family', 'is_classic', 'is_romantic', 'is_TV_MA', 'is_R',
       'is_PG_13', 'is_TV_14', 'is_TV_PG', 'is_PG', 'is_TV_G', 'is_TV_Y',
       'is_TV_Y7', 'is_G', 'is_NC_17', 'is_unrated']]

In [32]:
netflix_update_knn = pd.concat([newer_netflix_update_knn, dummified_cast], axis = 1)
netflix_update_knn.head(3)

,is_movie,is_english,is_modern,is_international,is_drama,is_horror,is_action_adventure,is_crime_mystery_thrillers,is_docu,is_indie,...,Çağatay Ulusoy,Çağlar Çorumlu,Çetin Tekindor,Öykü Karayel,Úrsula Corberó,İbrahim Büyükak,İbrahim Çelikkol,Şahin Irmak,Şükrü Özyıldız,Ṣọpẹ́ Dìrísù
title,,,,,,,,,,,,,,,,,,,,,
3%,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7:19,1,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23:59,1,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# STEP 8: Content-Based Recommender with a KNN Algorithm

In [33]:
from sklearn.neighbors import NearestNeighbors

In [34]:
from scipy.sparse import csr_matrix

In [35]:
netflix_update_knn_bool = netflix_update_knn.iloc[:, :].astype("bool")

In [36]:
netflix_bool_feature_matrix = csr_matrix(netflix_update_knn_bool.values)
print(netflix_bool_feature_matrix)

  (0, 2)	True
  (0, 3)	True
  (0, 4)	True
  (0, 14)	True
  (0, 19)	True
  (0, 2275)	True
  (1, 0)	True
  (1, 2)	True
  (1, 3)	True
  (1, 4)	True
  (1, 19)	True
  (1, 1162)	True
  (2, 0)	True
  (2, 2)	True
  (2, 3)	True
  (2, 5)	True
  (2, 20)	True
  (2, 4469)	True
  (3, 0)	True
  (3, 1)	True
  (3, 2)	True
  (3, 6)	True
  (3, 9)	True
  (3, 14)	True
  (3, 21)	True
  :	:
  (7783, 3)	True
  (7783, 4)	True
  (7783, 15)	True
  (7783, 22)	True
  (7783, 4680)	True
  (7784, 0)	True
  (7784, 2)	True
  (7784, 3)	True
  (7784, 8)	True
  (7784, 15)	True
  (7784, 19)	True
  (7784, 3327)	True
  (7785, 1)	True
  (7785, 2)	True
  (7785, 3)	True
  (7785, 10)	True
  (7785, 23)	True
  (7785, 105)	True
  (7786, 0)	True
  (7786, 1)	True
  (7786, 2)	True
  (7786, 8)	True
  (7786, 15)	True
  (7786, 19)	True
  (7786, 4649)	True


In [37]:
knn_model = NearestNeighbors(algorithm = 'brute', metric = 'cosine')
knn_model.fit(netflix_bool_feature_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [38]:
recommended = []

In [39]:
for each in netflix_update_knn_bool.index:
    
    distances, indices = knn_model.kneighbors(netflix_update_knn_bool.loc[each, :].values.reshape(1, -1), n_neighbors = 20)
    
    for num in range(0,len(distances.flatten())):  
        
        recommended.append([each, num, netflix_update_knn_bool.index[indices.flatten()[num]], distances.flatten()[num]])
    

In [40]:
netflix_recommendations = pd.DataFrame(recommended, columns = ["title", "recommendation_placement", "recommended_titles", "metric"])

In [41]:
netflix_recommendations = netflix_recommendations.drop_duplicates()

In [42]:
netflix_recommendations[(netflix_recommendations['title'] == "Step Brothers") & (netflix_recommendations['recommended_titles'] != "Step Brothers")].sort_values("metric")

,title,recommendation_placement,recommended_titles,metric
116180,Step Brothers,0,Casa de mi Padre,0.000000
116197,Step Brothers,17,A Bad Moms Christmas,0.166667
116196,Step Brothers,16,Guest House,0.166667
116195,Step Brothers,15,Road Trip: Beer Pong,0.166667
116194,Step Brothers,14,Drunk Parents,0.166667
116193,Step Brothers,13,The D Train,0.166667
116192,Step Brothers,12,Where's the Money,0.166667
116191,Step Brothers,11,Undercover Brother 2,0.166667
116198,Step Brothers,18,Wine Country,0.166667
116190,Step Brothers,10,Kevin Hart: Laugh at My Pain,0.166667


In [43]:
netflix_recommendations[(netflix_recommendations['title'] == "Lucifer") & (netflix_recommendations['recommended_titles'] != "Lucifer")].sort_values("metric")

,title,recommendation_placement,recommended_titles,metric
76141,Lucifer,1,The Good Cop,0.142857
76142,Lucifer,2,Battle Creek,0.142857
76143,Lucifer,3,Republic of Doyle,0.142857
76157,Lucifer,17,American Odyssey,0.228483
76156,Lucifer,16,Private Practice,0.228483
76155,Lucifer,15,Frequency,0.228483
76154,Lucifer,14,In The Dark,0.228483
76153,Lucifer,13,The Unicorn,0.228483
76152,Lucifer,12,NCIS,0.228483
76151,Lucifer,11,Criminal Minds,0.228483


In [44]:
netflix_recommendations[(netflix_recommendations['title'] == "Fairy Tail") & (netflix_recommendations['recommended_titles'] != "Fairy Tail")].sort_values("metric")

,title,recommendation_placement,recommended_titles,metric
41800,Fairy Tail,0,Saint Seiya: The Lost Canvas,1.110223e-16
41817,Fairy Tail,17,Megalobox,2.000000e-01
41816,Fairy Tail,16,Knights of Sidonia,2.000000e-01
41815,Fairy Tail,15,No Game No Life,2.000000e-01
41814,Fairy Tail,14,Black Butler,2.000000e-01
41813,Fairy Tail,13,Sword Art Online,2.000000e-01
41812,Fairy Tail,12,Sword Art Online Alternative: Gun Gale Online,2.000000e-01
41811,Fairy Tail,11,Dorohedoro,2.000000e-01
41818,Fairy Tail,18,Cagaster of an Insect Cage,2.000000e-01
41810,Fairy Tail,10,SWORDGAI The Animation,2.000000e-01


# STEP 9: Create all of this into a single function

In [45]:
def recommendations(title, num_recommendations):
    
    # Create a matrix where you'll be using to calculate the distance b/t each title
    some_matrix = csr_matrix(netflix_update_knn_bool.values)
    # Establish the unsupervised learner for generating recommendations ... use the brute force approach with appropriate metric
    knn_model = NearestNeighbors(algorithm = 'brute', metric = 'cosine')
    # Fit the nearest neighbors estimator from the training dataset
    knn_model.fit(some_matrix)
    
    # Have an empty list to populate with recommendations
    recommended = []
    
    # Iterate over each row in the matrix to get the actual distance metric and then populate the empty list with relevant into
    
    for each in netflix_update_knn_bool.index:
        distances, indices = knn_model.kneighbors(netflix_update_knn_bool.loc[each, :].values.reshape(1, -1), n_neighbors = num_recommendations)
    
        for num in range(0,len(distances.flatten())):  
            recommended.append([each, num, netflix_update_knn_bool.index[indices.flatten()[num]], distances.flatten()[num]])
    
    # Rename the columns of the recommendation matrix-turned data frame
    recommendations = pd.DataFrame(recommended, columns = ["title", "recommendation_placement", "recommended_titles", "metric"])
    # Remove duplicate entries 
    recommendations = recommendations.drop_duplicates()
    # REtrieve the recommendations 
    recommend = recommendations[(recommendations['title'] == title) & (recommendations['recommended_titles'] != title)].sort_values("metric")
    return recommend

In [48]:
recommendations("Lucifer", 15)

,title,recommendation_placement,recommended_titles,metric
57106,Lucifer,1,The Good Cop,0.142857
57107,Lucifer,2,Battle Creek,0.142857
57108,Lucifer,3,Republic of Doyle,0.142857
57109,Lucifer,4,The Lizzie Borden Chronicles,0.228483
57110,Lucifer,5,American Odyssey,0.228483
57111,Lucifer,6,Private Practice,0.228483
57112,Lucifer,7,Frequency,0.228483
57113,Lucifer,8,In The Dark,0.228483
57114,Lucifer,9,Criminal Minds,0.228483
57115,Lucifer,10,NCIS,0.228483


In [49]:
recommendations("The Other Guys", 10)

,title,recommendation_placement,recommended_titles,metric
67291,The Other Guys,1,Get Smart,0.142857
67292,The Other Guys,2,Ocean's Thirteen,0.142857
67293,The Other Guys,3,Ocean's Twelve,0.142857
67294,The Other Guys,4,Austin Powers in Goldmember,0.142857
67295,The Other Guys,5,Stranger than Fiction,0.142857
67296,The Other Guys,6,The Dukes of Hazzard,0.142857
67297,The Other Guys,7,Starsky & Hutch,0.142857
67298,The Other Guys,8,The Tuxedo,0.142857
67299,The Other Guys,9,Code Name: The Cleaner,0.142857
